In [1]:
import re
import os
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from collections import defaultdict

In [2]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [9]:
#REad the stop words list
stop_words_paht='data/reuters/stopwords'
stop_words=[]
with open(stop_words_paht,'r') as file:
    content=file.read().split('\n')
    stop_words.append(content)
    

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
data_directory = 'data/reuters/training/'
filenames=os.listdir(data_directory)
filenames.sort(key=lambda x: int(x))

In [11]:
stop_words_path = 'data/reuters/stopwords'
data_directory = 'data/reuters/training/'
filenames = os.listdir(data_directory)
filenames.sort(key=lambda x: int(x))
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))
processed_doc = {}
real_doc = {}

def load_data():
    for filename in filenames:
        with open(os.path.join(data_directory, filename), 'r') as file:
            content = file.read()
            real_doc[filename] = content
            content = normalize_text(content)
            words_list = []
            for word in content.translate(str.maketrans('', '', string.punctuation)).split(" "):
                if word not in stop_words[0] or word not in stop_words_nltk:
                    word = lemmatizer.lemmatize(word)
                    words_list.append(word)
            lemmatized_doc = " ".join(words_list)
            processed_doc[filename] = lemmatized_doc

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jeffersonc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
load_data()
filenames_list = []
original_text_list = []
processed_text = []

# Iterar sobre cada nombre de archivo y sus correspondientes textos originales y procesados
for filename, original_text in real_doc.items():
    filenames_list.append(filename)
    original_text_list.append(original_text)
    processed_text.append(processed_doc[filename])

# Crear el DataFrame
df = pd.DataFrame({
    'filename': filenames_list,
    'original_text': original_text_list,
    'processed_text': processed_text
})

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Creamos una instancia de CountVectorizer para BoW
count_vectorizer = CountVectorizer(binary=True)
# Creamos una instancia de CountVectorizer para TF-IDF
tfidf_vectorizer = TfidfVectorizer()
# Aplicar Bag of Words (BoW)
bow_matrix = count_vectorizer.fit_transform(df['processed_text'])
bow_feature_names = count_vectorizer.get_feature_names_out()
# Aplicar TF-IDF 
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_text']) 
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

In [16]:
query = "income"  # Ejemplo de consulta
query_bow = count_vectorizer.transform([query])  #vectorizamos al query para BoW

In [17]:
#Calculamos la similituds jaccard usando la matriz de BoW
def similitud_jaccard(query, text):
    interseccion = np.sum(np.logical_and(query, text))
    union = np.sum(np.logical_or(query, text))
    jaccard_score = interseccion / union if union != 0 else 0.0
    return jaccard_score

import time
start=time.time()
jaccard_similarities1 = []
for idx in range(bow_matrix.shape[0]):
    
    a=query_bow.toarray().squeeze()
    b=bow_matrix[idx].toarray().squeeze()
    similarity=similitud_jaccard(a,b)

    jaccard_similarities1.append(similarity)
end=time.time()
print(end-start)




1.3394887447357178


In [18]:
#Nos devuelve los indices de la lista de jaccard_similarities1
sorted_indices1 = np.argsort(jaccard_similarities1)[::-1]

In [20]:
i=0
for idx in sorted_indices1:
    filename=df['filename'].iloc[idx]
    texto = df['original_text'].iloc[idx]
    print(f"Texto #{filename} - Similitud Jaccard: {jaccard_similarities1[idx]}")
    print(texto)
    print("-----------------------------------------------------")
    if i>5:
        break
    else:
        i+=1

Texto #7098 - Similitud Jaccard: 0.16666666666666666
U.S. PERSONAL INCOME ROSE 0.9 PCT IN FEBRUARY, SPENDING UP 1.7 PCT

  U.S. PERSONAL INCOME ROSE 0.9 PCT IN FEBRUARY, SPENDING UP 1.7 PCT
  


-----------------------------------------------------
Texto #8016 - Similitud Jaccard: 0.08333333333333333
MONTGOMERY STREET INCOME &lt;MTS> MONTHLY DIVIDEND
  Mthly div 15 cts vs 15 cts
      Pay April 15
      Record April 1
  


-----------------------------------------------------
Texto #508 - Similitud Jaccard: 0.07692307692307693
&lt;FRANKLIN CALIFORNIA TAX-FREE INCOME FUND>PAYOUT
  Mthly div 4.5 cts vs 4.5 cts prior
      Pay March 13
      Record March Two
  


-----------------------------------------------------
Texto #512 - Similitud Jaccard: 0.07142857142857142
&lt;FRANKLIN FEDERAL TAX-FREE INCOME FUND> PAYOUT
  Mthly div 7.7 cts vs 7.7 cts prior
      Pay March 13
      Record March Two
  


-----------------------------------------------------
Texto #513 - Similitud Jaccard: 0.066

In [21]:
query_tfidf =tfidf_vectorizer.transform([query]) #vectorizamos a tf-idf la query

# Cosine similarity using numpy
def cosine_sim(a,b):
    if np.linalg.norm(a)*np.linalg.norm(b)>0.0:        
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    else:
        return 0.0
    
cosine_distances_list = []
for idx in range(bow_matrix.shape[0]):
    distance=cosine_sim(bow_matrix[idx].toarray().squeeze(),query_tfidf.toarray().squeeze())
    
    cosine_distances_list.append(distance)
    
    
sorted_indices_cos = np.argsort(cosine_distances_list)[::-1]


i=0
print("Query: ",query)
for i, idx2 in enumerate(sorted_indices_cos):
    if (cosine_distances_list[idx2]>0.0 and i<5):
        filename=df['filename'].iloc[idx2]
        texto = df['original_text'].iloc[idx2]
        print(f"Texto #{filename} - Distancia coseno: {cosine_distances_list[idx2]}")
        print(texto)
        print("-----------------------------------------------------")
    else:
        break
   

Query:  income
Texto #7098 - Distancia coseno: 0.4082482904638631
U.S. PERSONAL INCOME ROSE 0.9 PCT IN FEBRUARY, SPENDING UP 1.7 PCT

  U.S. PERSONAL INCOME ROSE 0.9 PCT IN FEBRUARY, SPENDING UP 1.7 PCT
  


-----------------------------------------------------
Texto #8016 - Distancia coseno: 0.2886751345948129
MONTGOMERY STREET INCOME &lt;MTS> MONTHLY DIVIDEND
  Mthly div 15 cts vs 15 cts
      Pay April 15
      Record April 1
  


-----------------------------------------------------
Texto #508 - Distancia coseno: 0.2773500981126146
&lt;FRANKLIN CALIFORNIA TAX-FREE INCOME FUND>PAYOUT
  Mthly div 4.5 cts vs 4.5 cts prior
      Pay March 13
      Record March Two
  


-----------------------------------------------------
Texto #512 - Distancia coseno: 0.2672612419124244
&lt;FRANKLIN FEDERAL TAX-FREE INCOME FUND> PAYOUT
  Mthly div 7.7 cts vs 7.7 cts prior
      Pay March 13
      Record March Two
  


-----------------------------------------------------
Texto #513 - Distancia coseno:

In [22]:
archivo_resultados = 'data/reuters/cats.txt'

consultas_resultados = {}

with open(archivo_resultados, 'r') as file:
    for line in file:
        line = line.strip()
        
        if line.startswith('training'):
            parts = line.split()
            archivo = parts[0]
            archivo= archivo.split('/')[1].strip().split()[0]
            consulta = ' '.join(parts[1:])
            if consulta in consultas_resultados:
                consultas_resultados[consulta].add(archivo)
            else:
                consultas_resultados[consulta] = {archivo}
                


In [23]:
def query_to_vectorBow(query):
    return count_vectorizer.transform([query])
    
    
def similitud_jaccard(query, text):
    interseccion = np.sum(np.logical_and(query, text))
    union = np.sum(np.logical_or(query, text))
    jaccard_score = interseccion / union if union != 0 else 0.0
    return jaccard_score


def calculate_jaccard(query):
    #start=time.time()
    jaccard_similarities1 = []
    best_titles_jaccard=[]
    query_bow=query_to_vectorBow(query)
    for idx in range(bow_matrix.shape[0]):        
        a=query_bow.toarray().squeeze()
        b=bow_matrix[idx].toarray().squeeze()
        similarity=similitud_jaccard(a,b)

        jaccard_similarities1.append(similarity)
    sorted_indices = np.argsort(jaccard_similarities1)[::-1]
    for idx in sorted_indices:
        if jaccard_similarities1[idx]>0.0:
            filename=df['filename'].iloc[idx]
            best_titles_jaccard.append(filename)
    return best_titles_jaccard
        
    #end=time.time()
    #print(end-start)
    

def get_true_positives(predicted, truth):
    true_positives = 0
    for value in predicted:
        if value in truth:
            true_positives += 1

    #print("True positives", true_positives)
    return true_positives


def get_false_negatives(predicted, truth):
    set_predicted = set(predicted)
    set_verdaderos = set(truth)
    false_negatives_list = list(set_verdaderos-set_predicted)
    false_negatives = len(false_negatives_list)

    #print("False negatives", false_negatives)
    return false_negatives


def get_false_positive(predicted, truth):
    set_predicted = set(predicted)
    set_verdaderos = set(truth)
    false_positives_list = list(set_predicted-set_verdaderos)
    false_positives = len(false_positives_list)
    #print("False positives", false_positives)

    return false_positives


In [24]:
#Para jaccard usando BoW
recall_results=[]
precision_results=[]
f1_score_results=[]
lemmatizer = WordNetLemmatizer()
for key in consultas_resultados:
    truth=consultas_resultados[key]
    key_lematized=lemmatizer.lemmatize(key)
    best_titles_jaccard=calculate_jaccard(key_lematized)
    true_positives=get_true_positives(best_titles_jaccard,truth)
    false_negatives=get_false_negatives(best_titles_jaccard,truth)
    false_positives=get_false_positive(best_titles_jaccard,truth)
    if true_positives + false_negatives == 0:
        recall = 0  
    else:
        recall = true_positives / (true_positives + false_negatives)
    
    recall_results.append(recall)
    
    if true_positives + false_positives == 0:
        precision = 0  
    else:
        precision = true_positives / (true_positives + false_positives)
    
    precision_results.append(precision)
    
    if ((2*true_positives)+false_negatives+false_positives)==0:
        f1_score=0
    else:
        f1_score=(2*true_positives)/((2*true_positives)+false_negatives+false_positives)
    f1_score_results.append(f1_score)
    

promedio_recall=sum(recall_results)/len(recall_results)
promedio_precision=sum(precision_results)/len(precision_results)    
promedio_f1_score=sum(f1_score_results)/len(f1_score_results)

print(f"Promedio de Recall jaccard: {promedio_recall}")
print(f"Promedio de Precisión jaccard: {promedio_precision}")
print(f"Promedio f1 score jaccard: {promedio_f1_score}")    

Promedio de Recall jaccard: 0.8378790622708628
Promedio de Precisión jaccard: 0.03297630019929637
Promedio f1 score jaccard: 0.04514864039827736


In [25]:
# Cosine similarity using numpy
def cosine_sim(a,b):
    if np.linalg.norm(a)*np.linalg.norm(b)>0.0:        
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    else:
        return 0.0
    
    
def calculate_cosine(query):
    #start=time.time()
    distances = []
    best_titles_cosine=[]
    query_bow=query_to_vectorBow(query)
    for idx in range(bow_matrix.shape[0]):        
        a=query_bow.toarray().squeeze()
        b=bow_matrix[idx].toarray().squeeze()
        similarity=cosine_sim(a,b)

        distances.append(similarity)
    sorted_indices = np.argsort(jaccard_similarities1)[::-1]
    for idx in sorted_indices:
        if distances[idx]>0.0:
            filename=df['filename'].iloc[idx]
            best_titles_cosine.append(filename)
    return best_titles_cosine

In [26]:
#Para TF-IDF con cosine
recall_results=[]
precision_results=[]
f1_score_results=[]
lemmatizer = WordNetLemmatizer()
for key in consultas_resultados:
    truth=consultas_resultados[key]
    key_lematized=lemmatizer.lemmatize(key)
    best_titles=calculate_cosine(key_lematized)
    true_positives=get_true_positives(best_titles,truth)
    false_negatives=get_false_negatives(best_titles,truth)
    false_positives=get_false_positive(best_titles,truth)
    if true_positives + false_negatives == 0:
        recall = 0  
    else:
        recall = true_positives / (true_positives + false_negatives)
    
    recall_results.append(recall)
    
    if true_positives + false_positives == 0:
        precision = 0  
    else:
        precision = true_positives / (true_positives + false_positives)
    
    precision_results.append(precision)
    
    if ((2*true_positives)+false_negatives+false_positives)==0:
        f1_score=0
    else:
        f1_score=(2*true_positives)/((2*true_positives)+false_negatives+false_positives)
    f1_score_results.append(f1_score)
    

promedio_recall=sum(recall_results)/len(recall_results)
promedio_precision=sum(precision_results)/len(precision_results)    
promedio_f1_score=sum(f1_score_results)/len(f1_score_results)

print(f"Promedio de Recall cosine: {promedio_recall}")
print(f"Promedio de Precisión cosine: {promedio_precision}")
print(f"Promedio f1 score cosine: {promedio_f1_score}")
    